Approche : mélanger analyse lexicale et par apprentissage.

1. Analyse lexicale des tweets :
    - POS tagging, lemmisation
    - Traduction des mots en Anglais pour pouvoir utiliser Sentiwordnet et obtenir la polarité des mots

2. Application d'un modèle d'apprentissage supervisé :
    - HMM, SVM, etc...

Liste des POS tags français : http://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/french-tagset.html

Etudier la possibilité d'ajout de lexiques d'opinion en Français : http://alpage.inria.fr/~sagot/wolf.html, http://sites.univ-provence.fr/wpsycle/outils_recherche/liwc/FrenchLIWCDictionary_V1_1.dic, http://sites.univ-provence.fr/~wpsycle/outils_recherche/outils_recherche.html#emotaix

Voir l'approche compositionnelle : Moilanen 2007

In [93]:
import pprint
import treetaggerwrapper
import time
import gc

import pymongo as pym
#import nltk.data
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
import stop_words
#from nltk.stem import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np

In [67]:
def mongo_to_df(collection, n_last_tweets=0, retweet=False):
    tweets = collection.find(filter={'text':{'$exists':True}}, 
                             projection={'_id':False}).sort('$natural',-1).limit(n_last_tweets)
    df = pd.DataFrame()
    listTweets, listCandidats, listSentiments = [], [], []
    
    for t in tweets: 
        if not retweet: # filtrage des retweets
            if 'rt @' in t['text']:
                continue

        if t['text']: # test si liste non vide
            listTweets.append(t['text'])
            try:
                listCandidats.append(t['candidat'])
            except:
                listCandidats.append(None)
            
            try:
                listSentiments.append(t['sentiment'])
            except:
                listSentiments.append(None)
    
    df['text'], df['candidat'], df['sentiment'] = listTweets, listCandidats, listSentiments
    return df


def load_tweets(client, spellcheck=True, label_auto=True, retweet=True):
    
    if spellcheck:
        collection = client.tweet.spellchecked
        df_tweets = mongo_to_df(collection, n_last_tweets=0, retweet=retweet)
        print('Correction orthographique activée.')
        if not label_auto:
            df_tweets = df_tweets[:client.tweet.train.count()]
    else:
        collection = client.tweet.train
        df_tweets = mongo_to_df(collection, n_last_tweets=0, retweet=retweet)
        print('Correction orthographique desactivée.')
        if label_auto:
            # Base annotée automatiquement, sur la base des hashtags (uniquement des tweets positifs)
            collection = client.tweet.labelised
            df_tweets_auto = mongo_to_df(collection, n_last_tweets=0, retweet=retweet)
            print('Ajout des tweets labélisés automatiquement...')
            print('{} tweets ajoutés.'.format(df_tweets_auto['text'].count()))
            df_tweets = pd.concat([df_tweets, df_tweets_auto], axis=0, ignore_index=True)
        
    print('\n{} tweets au total récupérés pour entraînement, répartis comme suit :'.format(df_tweets['text'].count()))
    print(df_tweets['sentiment'].value_counts())
    print(df_tweets['candidat'].value_counts())
        
    return df_tweets


def process_texts(list_of_texts, pos_tag_list, stop_words):
    # Processing the tweets (POS tagging, lemmatization, spellchecking)
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')
    list_of_processed_texts = []
    
    for text in list_of_texts:
        # Etape de filtrage
        text = re.sub(r'\w*…', '', text) # mot tronqué par Twitter
        text = re.sub(r'(?:htt)\S*', '', text) # retrait des liens http
        text = re.sub(r'\n', ' ', text) # retrait des sauts de ligne
        text = re.sub(r'\xad', '-', text)
        text = re.sub(r'@\w*', '', text) # retrait des mentions @ (ne détecte pas @XXX@...)
        text = re.sub(r'\.{3,}', '...', text) # ....... => points de suspension
        text = re.sub(r'(?=\.\w)(\.)', '. ', text) # remplacer un point entre deux mots 'A.B' par 'A. B'
        #text = re.sub(r'\[a-zA-Z]*(?!\S)', '', text) # retrait de ce qui n'est pas un mot
        #text = re.sub(r'^rt.*: ', '', text) # retrait de la mention retweet
        #text = re.sub(r'\d', '', text) # retrait des chiffres
        #text = re.sub(r',;!?\/\*(){}«»', ' ', text)
        #text = re.sub('|'.join(['’', '_', '/', '-', '\'', '“', '\.']), ' ', text)
        #text = re.sub('|'.join([elem + '\'' for elem in 'cdjlmnst']), '', text) # apostrophes
        
        # TODO: (optionnel) retirer les # avant d'utiliser TreeTagger
        # puis les remettre avec le tag HASH|
        
        # TODO: correction orthographique des tweets
        
        tags = tagger.tag_text(text)
        
        try:
            tagged_text = ['{}|{}'.format(t.split('\t')[1], t.split('\t')[2]) for t in tags
                           if (t.split('\t')[2] not in stop_words
                               and t.split('\t')[1] in pos_tag_list)]
        except:
            tagged_text = ['ERREUR']
        
        # append les HASH|#...
        # TODO: (optionnel) gérer les accents sur les mots
        list_of_processed_texts.append(tagged_text)
        
    return list_of_processed_texts

In [146]:
def build_Xy(df_tweets, pos_tags_to_keep, stop_words):
    # Tweet feature extraction
    # TODO: faire des tests en ajoutant/retirant des features
    hashtag = np.array([t.count('#') / 1. for t in df_tweets['text']])
    links = np.array([t.count('http') / 1. for t in df_tweets['text']])
    at = np.array([t.count('@') / 1. for t in df_tweets['text']])
    n_car = np.array([np.log(len(t))/4 / 1. for t in df_tweets['text']])
    n_words = np.array([np.log(len(t.split(' '))) / 2 for t in df_tweets['text']])
    n_2points = np.array([t.count(':') / 1. for t in df_tweets['text']])
    n_exc = np.array([t.count('!') / 1. for t in df_tweets['text']])
    n_int = np.array([t.count('?') / 1. for t in df_tweets['text']])
    n_quotes = np.array([(t.count('"') + t.count('»')) / 2 for t in df_tweets['text']])

    print('Tagging des tweets en cours...')
    tweet_list = process_texts(df_tweets['text'], pos_tags_to_keep, stop_words)
    #TODO: concatener la liste au df. Retirer les doublons du df avec drop_duplicates.
    # Recreer la liste des tweets sans doublons à partir de la colonne

    print('TreeTagger a renvoyé {} erreur(s).'.format(tweet_list.count('ERREUR')))

    # Building feature matrix
    # TODO: jouer sur les min et max df
    print('\nCréation de la matrice de features...')
    vectorizer = TfidfVectorizer(strip_accents=None, analyzer='word', decode_error='strict',
                                use_idf=False, norm=None, binary=False, min_df=4, max_df=1.0, ngram_range=(1,2))
    
    tfidf = vectorizer.fit_transform([' '.join(tweet) for tweet in tweet_list])
    X = pd.DataFrame(tfidf.toarray())

    X_added_features = pd.DataFrame(data={'#': hashtag,
                                          #'http': links,
                                          '@': at,
                                          'n_car': n_car,
                                          'n_words': n_words,
                                          ':': n_2points,
                                          '!': n_exc,
                                          '?': n_int,
                                          '""': n_quotes
                                         })

    X = pd.concat([X, X_added_features], axis=1)
    y = df_tweets['sentiment']
    
    Xy = pd.concat([X, y], axis=1)
    Xy.drop_duplicates(inplace=True)
    print('{} doublons retirés ({} documents restants).'.format(X.shape[0] - Xy.shape[0], Xy.shape[0]))
    print(Xy[:5])

    return Xy.drop('sentiment', axis=1), Xy['sentiment']

In [132]:
def find_best_params(X, y, model, params_to_test, cut=1):
    if model not in ['logistic', 'svc', 'nb', 'rf']:
        print('Il faut choisir un modèle parmi logistic, svc, rf et nb.')
        return
    
    # Building train & test sets
    X_train, X_test, y_train, y_test = train_test_split(X[:-cut], y[:-cut], test_size = 0.15)
    X_train = pd.concat([X_train, X[-cut:]], axis=0)
    y_train = pd.concat([y_train, y[-cut:]], axis=0)
    
    # mélange des lignes
    rng = np.random.randint(1000)
    X_train = X_train.sample(frac=1.0, replace=False, random_state=rng)
    y_train = y_train.sample(frac=1.0, replace=False, random_state=rng)
    n_samples, vocabulaire = X.shape

    print("Répartition dans le dataset de train ({} tweets) : \n".format(len(y_train)),
          '\tNégatif : {:.1f}'.format(len(np.extract(y_train == -1, y_train)) / len(y_train) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_train == 1, y_train)) / len(y_train) * 100), '%')
    print("Répartition dans le dataset de test ({} tweets) : \n".format(len(y_test)),
          '\tNégatif : {:.1f}'.format(len(np.extract(y_test == -1, y_test)) / len(y_test) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_test == 1, y_test)) / len(y_test) * 100), '%')
    print('Tweets : ' + str(n_samples) + ' / ' + 'N-grams : ' + str(vocabulaire))

    # Choice of models
    if model == 'logistic':
        clf = LogisticRegression(max_iter=4000, class_weight='balanced', multi_class='ovr')
    if model == 'svc':
        clf = LinearSVC(class_weight='balanced')
    if model == 'nb' :
        clf = MultinomialNB()
    if model == 'rf' :
        clf = RandomForestClassifier(criterion='gini', max_depth=None, max_features='auto',
                                     bootstrap=True, n_jobs=-1, verbose=0, class_weight='balanced_subsample')

    gcv = GridSearchCV(clf, params_to_test, verbose=9, n_jobs=-1, cv=4, refit=True)
    gcv.fit(X_train, y_train)
    print('Les meilleurs paramètres pour {} sont {}.'.format(model, gcv.best_params_))
    
    # Fit & predict
    print('Prédiction sur l\'ensemble de test avec ces paramètres...')
    y_pred = gcv.predict(X_test)

    print('Score', np.sum(y_pred == y_test) / len(y_pred))
    print('Répartition des prédictions : \n',
          '\tNégatif : {:.1f}'.format(len(np.extract(y_pred == -1, y_pred)) / len(y_pred) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_pred == 1, y_pred)) / len(y_pred) * 100), '%')

    # matrice de confusion
    cf = confusion_matrix(y_test, y_pred)
    recall = [cf[i,i]/cf[i,:].sum() for i in range(3)]
    precision = [cf[i,i]/cf[:,i].sum() for i in range(3)]
    print('\nMatrice de confusion (ligne: classe réelle, colonne: classe prédite):')
    print(cf)
    print('Recall (négatif, neutre, positif) : {:.3f}, {:.3f}, {:.3f}'.format(recall[0], recall[1], recall[2]))
    print('Précision (négatif, neutre, positif) : {:.3f}, {:.3f}, {:.3f}'.format(precision[0], precision[1], precision[2]))
    
    return

def fit_predict(X, y, clf, cut=1):
    
    # Building train & test sets
    X_train, X_test, y_train, y_test = train_test_split(X[:-cut], y[:-cut], test_size = 0.15)
    X_train = pd.concat([X_train, X[-cut:]], axis=0)
    y_train = pd.concat([y_train, y[-cut:]], axis=0)
    
    # mélange des lignes
    rng = np.random.randint(1000)
    X_train = X_train.sample(frac=1.0, replace=False, random_state=rng)
    y_train = y_train.sample(frac=1.0, replace=False, random_state=rng)
    n_samples, vocabulaire = X.shape

    print("Répartition dans le dataset de train ({} tweets) : \n".format(len(y_train)),
          '\tNégatif : {:.1f}'.format(len(np.extract(y_train == -1, y_train)) / len(y_train) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_train == 1, y_train)) / len(y_train) * 100), '%')
    print("Répartition dans le dataset de test ({} tweets) : \n".format(len(y_test)),
          '\tNégatif : {:.1f}'.format(len(np.extract(y_test == -1, y_test)) / len(y_test) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_test == 1, y_test)) / len(y_test) * 100), '%')
    print('Tweets : ' + str(n_samples) + ' / ' + 'N-grams : ' + str(vocabulaire))

    
    # Fit & predict
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print('Score', np.sum(y_pred == y_test) / len(y_pred))
    print('Répartition des prédictions : \n',
          '\tNégatif : {:.1f}'.format(len(np.extract(y_pred == -1, y_pred)) / len(y_pred) * 100),
          '%\n\tPositif : {:.1f}'.format(len(np.extract(y_pred == 1, y_pred)) / len(y_pred) * 100), '%')

    # matrice de confusion
    cf = confusion_matrix(y_test, y_pred)
    recall = [cf[i,i]/cf[i,:].sum() for i in range(3)]
    precision = [cf[i,i]/cf[:,i].sum() for i in range(3)]
    print('\nMatrice de confusion (ligne: classe réelle, colonne: classe prédite):')
    print(cf)
    print('Recall (négatif, neutre, positif) : {:.3f}, {:.3f}, {:.3f}'.format(recall[0], recall[1], recall[2]))
    print('Précision (négatif, neutre, positif) : {:.3f}, {:.3f}, {:.3f}'.format(precision[0], precision[1], precision[2]))
    
    return

#### Test du TreeTagger

In [70]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')
tags = tagger.tag_text('mdr lol : test nombre 800,000 €')
print('Output du tagger :', tags)
tagged_text = ['{}|{}'.format(t.split('\t')[1], t.split('\t')[2]) for t in tags]
print('Création des features :', tagged_text)

Output du tagger : ['mdr\tNOM\tmdr', 'lol\tNOM\tlol', ':\tPUN\t:', 'test\tNOM\ttest', 'nombre\tNOM\tnombre', '800\tNUM\t@card@', ',\tPUN\t,', '000\tNUM\t@card@', '€\tNOM\t€']
Création des features : ['NOM|mdr', 'NOM|lol', 'PUN|:', 'NOM|test', 'NOM|nombre', 'NUM|@card@', 'PUN|,', 'NUM|@card@', 'NOM|€']


#### Echantillons de tweets dans la base

In [82]:
client = pym.MongoClient('localhost',27017)
df_labelised = mongo_to_df(client.tweet.labelised, retweet=True)
df_spell = mongo_to_df(client.tweet.spellchecked, retweet=True)
df_train = mongo_to_df(client.tweet.train, retweet=True)
print(df_spell.shape[0], df_train.shape[0], df_labelised.shape[0])
df_labelised['sentiment'].value_counts()

0 10001 8893


 0    3390
-1    3233
 1    2270
Name: sentiment, dtype: int64

In [72]:
for i in range(0, 20):
    print(df_spell['text'][i])
    print(2*'-')
    print(df_train['text'][i])
    print(10*'-')

IndexError: index out of bounds

### Chargement des tweets depuis Mongo et création des features

In [83]:
# Choix de la base à utiliser
client = pym.MongoClient('localhost',27017)
df = load_tweets(client, spellcheck=False, label_auto=True, retweet=False)

Correction orthographique desactivée.
Ajout des tweets labélisés automatiquement...
8887 tweets ajoutés.

16432 tweets au total récupérés pour entraînement, répartis comme suit :
-1.0    6972
 0.0    6234
 1.0    3226
Name: sentiment, dtype: int64
fillon       2707
macron       2542
le pen       2136
hamon        1044
melenchon     239
mélenchon     219
Name: candidat, dtype: int64


### Jouer avec les paramètres : POS tag à garder, dictionnaire de stop words

In [128]:
# Choix des POS tags à conserver
all_postags = ['ABR', 'ADJ', 'ADV', 'DET:ART', 'DET:POS', 'INT', 'KON', 'NAM', 'NOM', 'NUM', 'PRO',
                   'PRO:DEM', 'PRO:IND', 'PRO:PER', 'PRO:POS', 'PRO:REL', 'PRP', 'PRP:det', 'PUN', 'PUN:cit',
                   'SENT', 'SYM', 'VER:cond', 'VER:futu', 'VER:impe', 'VER:impf', 'VER:pper', 'VER:ppre',
                   'VER:pres', 'VER:simp', 'VER:subi', 'VER:subp']

pos_tags_to_keep = ['ADJ', 'ADV', 'NOM', 'NUM', 'PUN:cit', 'PRO:POS', 'PRO:DEM',
                    'VER:cond', 'VER:futu', 'VER:impe', 'VER:impf',
                    'VER:pper', 'VER:ppre', 'VER:pres', 'VER:simp', 'VER:subi', 'VER:subp']

# Choix des stop words
# stops = set(['rt','ds','qd','ss','ns','vs','nn','amp','gt','gd','gds','tt','pr','ac','mm', 'qu',
#             '``', 'ca', 'mdr', 'lol', 'dsl', 'cad']
#             + list('@ن%£€‘:&;')
#             + list('abcdefghijklmnopqrstuvwxyzà')
#            + stop_words.get_stop_words(language='fr')
#            + stopwords.words('french')
#            )
stops = set(list('abcdefghijklmnopqrstuvwxyz')
#           + ['rt','ds','qd','ss','ns','vs','nn','amp','gt','gd','gds','tt','pr','ac','mm','qu','dsl','cad']
           )
# TODO: améliorer la liste de stop words

X, y = build_Xy(df, pos_tags_to_keep, stops)

Tagging des tweets en cours...
TreeTagger a renvoyé 0 erreur(s).

Création de la matrice de features...
1623 doublons retirés (14809 documents restants).
     0    1    2    3    4    5    6    7    8    9    ...      9837    !  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0  2.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       0.0  0.0   

    ""    #    :    ?    @     n_car   n_words  sentiment  
0  0.0  1.0  1.0  0.0  0.0  1.228164  1.522261        1.0  
1  0.0  0.0  0.0  0.0  0.0  0.972955  0.972955       -1.0  
2  0.0  0.0  1.0  0.0  0.0  1.235411  1.609438       -1.0  
3  0.0  1.0  1.0  0.0  1.0  1.235411  1.545521       -1.0  
4  0.0  0.0  1.0  2.0  0.0  1.170533  1.242453       -1.0  

[5 rows x 9847 columns

### Test avec la regression logistique

In [129]:
params = {'penalty':['l2'], 'dual': [True], 'C' : [.5]}
find_best_params(X, y, 'logistic', params, cut=8000)

Répartition dans le dataset de train (13787 tweets) : 
 	Négatif : 45.7 %
	Positif : 21.6 %
Répartition dans le dataset de test (1022 tweets) : 
 	Négatif : 50.4 %
	Positif : 12.4 %
Tweets : 14809 / N-grams : 9846
Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:   43.6s remaining:   43.6s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   44.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   44.3s finished


Les meilleurs paramètres pour logistic sont {'C': 0.5, 'dual': True, 'penalty': 'l2'}.
Prédiction sur l'ensemble de test avec ces paramètres...
Score 0.634050880626
Répartition des prédictions : 
 	Négatif : 51.9 %
	Positif : 6.6 %

Matrice de confusion (ligne: classe réelle, colonne: classe prédite):
[[367 127  21]
 [103 256  21]
 [ 60  42  25]]
Recall (négatif, neutre, positif) : 0.713, 0.674, 0.197
Précision (négatif, neutre, positif) : 0.692, 0.602, 0.373


In [133]:
clf = LogisticRegression(C=.5, max_iter=2000, class_weight='balanced', multi_class='ovr',
                        penalty='l2', dual=True)
fit_predict(X, y, clf, cut=8000)

Répartition dans le dataset de train (13787 tweets) : 
 	Négatif : 45.6 %
	Positif : 21.7 %
Répartition dans le dataset de test (1022 tweets) : 
 	Négatif : 51.3 %
	Positif : 11.7 %
Tweets : 14809 / N-grams : 9846
Score 0.617416829746
Répartition des prédictions : 
 	Négatif : 50.8 %
	Positif : 6.4 %

Matrice de confusion (ligne: classe réelle, colonne: classe prédite):
[[363 140  21]
 [114 244  20]
 [ 42  54  24]]
Recall (négatif, neutre, positif) : 0.693, 0.646, 0.200
Précision (négatif, neutre, positif) : 0.699, 0.557, 0.369


### Test avec le LinearSVC

In [141]:
params = {'C' : [.02, .1, 1.]}
find_best_params(X, y, 'svc', params, cut=5000)

Répartition dans le dataset de train (13337 tweets) : 
 	Négatif : 44.7 %
	Positif : 21.7 %
Répartition dans le dataset de test (1472 tweets) : 
 	Négatif : 57.8 %
	Positif : 14.7 %
Tweets : 14809 / N-grams : 9846
Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  1.4min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  1.5min remaining:   29.5s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.8min finished


Les meilleurs paramètres pour svc sont {'C': 0.02}.
Prédiction sur l'ensemble de test avec ces paramètres...
Score 0.695652173913
Répartition des prédictions : 
 	Négatif : 56.6 %
	Positif : 8.5 %

Matrice de confusion (ligne: classe réelle, colonne: classe prédite):
[[654 170  27]
 [113 282  10]
 [ 66  62  88]]
Recall (négatif, neutre, positif) : 0.769, 0.696, 0.407
Précision (négatif, neutre, positif) : 0.785, 0.549, 0.704


In [ ]:
clf = LinearSVC(C=.02, class_weight='balanced')
fit_predict(X, y, clf, cut=8000)

### Test avec Naive Bayes

In [142]:
params = {'alpha': [.75, 1., 1.25]}
find_best_params(X, y, 'nb', params, cut=5000)

Répartition dans le dataset de train (13337 tweets) : 
 	Négatif : 44.8 %
	Positif : 21.6 %
Répartition dans le dataset de test (1472 tweets) : 
 	Négatif : 56.5 %
	Positif : 15.0 %
Tweets : 14809 / N-grams : 9846
Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   26.0s remaining:   18.5s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   29.8s remaining:    9.9s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   34.9s finished


Les meilleurs paramètres pour nb sont {'alpha': 1.0}.
Prédiction sur l'ensemble de test avec ces paramètres...
Score 0.677989130435
Répartition des prédictions : 
 	Négatif : 55.0 %
	Positif : 14.5 %

Matrice de confusion (ligne: classe réelle, colonne: classe prédite):
[[619 149  63]
 [121 264  35]
 [ 69  37 115]]
Recall (négatif, neutre, positif) : 0.745, 0.629, 0.520
Précision (négatif, neutre, positif) : 0.765, 0.587, 0.540


In [143]:
clf = MultinomialNB(alpha=1.)
fit_predict(X, y, clf, cut=5000)

Répartition dans le dataset de train (13337 tweets) : 
 	Négatif : 44.9 %
	Positif : 21.6 %
Répartition dans le dataset de test (1472 tweets) : 
 	Négatif : 55.6 %
	Positif : 15.1 %
Tweets : 14809 / N-grams : 9846
Score 0.674592391304
Répartition des prédictions : 
 	Négatif : 56.3 %
	Positif : 13.2 %

Matrice de confusion (ligne: classe réelle, colonne: classe prédite):
[[617 152  50]
 [136 263  32]
 [ 76  33 113]]
Recall (négatif, neutre, positif) : 0.753, 0.610, 0.509
Précision (négatif, neutre, positif) : 0.744, 0.587, 0.579


### Test avec Random Forest

In [144]:
params = {'n_estimators': [50, 60]}
find_best_params(X, y, 'rf', params, cut=5000)

Répartition dans le dataset de train (13337 tweets) : 
 	Négatif : 44.8 %
	Positif : 21.7 %
Répartition dans le dataset de test (1472 tweets) : 
 	Négatif : 56.3 %
	Positif : 14.3 %
Tweets : 14809 / N-grams : 9846
Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  1.6min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  1.8min remaining:   36.7s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.2min finished


Les meilleurs paramètres pour rf sont {'n_estimators': 50}.
Prédiction sur l'ensemble de test avec ces paramètres...
Score 0.697690217391
Répartition des prédictions : 
 	Négatif : 67.6 %
	Positif : 6.7 %

Matrice de confusion (ligne: classe réelle, colonne: classe prédite):
[[707 114   8]
 [197 233   3]
 [ 91  32  87]]
Recall (négatif, neutre, positif) : 0.853, 0.538, 0.414
Précision (négatif, neutre, positif) : 0.711, 0.615, 0.888


In [145]:
clf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None, max_features='auto',
                                     bootstrap=True, n_jobs=-1, verbose=0, class_weight='balanced')
fit_predict(X, y, clf, cut=5000)

Répartition dans le dataset de train (13337 tweets) : 
 	Négatif : 44.9 %
	Positif : 21.7 %
Répartition dans le dataset de test (1472 tweets) : 
 	Négatif : 56.0 %
	Positif : 14.2 %
Tweets : 14809 / N-grams : 9846
Score 0.701766304348
Répartition des prédictions : 
 	Négatif : 65.6 %
	Positif : 5.6 %

Matrice de confusion (ligne: classe réelle, colonne: classe prédite):
[[700 119   6]
 [167 264   7]
 [ 98  42  69]]
Recall (négatif, neutre, positif) : 0.848, 0.603, 0.330
Précision (négatif, neutre, positif) : 0.725, 0.621, 0.841
